<a href="https://colab.research.google.com/github/DeepLabCut/DeepLabCut/blob/master/examples/COLAB/COLAB_YOURDATA_TrainNetwork_VideoAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut Toolbox - Colab for standard (single animal) projects!
https://github.com/DeepLabCut/DeepLabCut

This notebook illustrates how to use the cloud to:
- create a training set
- train a network
- evaluate a network
- create simple quality check plots
- analyze novel videos!

###This notebook assumes you already have a project folder with labeled data! 

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.


Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [1]:
#(this will take a few minutes to install all the dependences!)
!pip install deeplabcut

     |████████████████████████████████| 542 kB 5.3 MB/s 
     |████████████████████████████████| 109 kB 35.7 MB/s 
     |████████████████████████████████| 352 kB 39.4 MB/s 
     |████████████████████████████████| 177 kB 44.8 MB/s 
     |████████████████████████████████| 9.8 MB 28.5 MB/s 
     |████████████████████████████████| 948 kB 31.8 MB/s 
     |████████████████████████████████| 29.2 MB 1.6 MB/s 
     |████████████████████████████████| 47.6 MB 65 kB/s 
     |████████████████████████████████| 296 kB 48.2 MB/s 
     |████████████████████████████████| 546 kB 47.4 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=1e5fb5f4ccd959d9031cb2e199b5f9815452c5900a1a1ad133e2a87bf4330063
  Stored in directory: /root/.cache/pip/wheels/ce/e0/ee/a2b3c5caab3418c1ccd8c4de573d4cbe13315d7e8b0a55fbc2
Successfully built filterpy
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.3
    Uninstalling scikit-image-0.18.3:
    

**(Be sure to click "RESTART RUNTIME" is it is displayed above above before moving on !)**

In [1]:
# Use TensorFlow 1.x:
%tensorflow_version 1.x

TensorFlow 1.x selected.


## Link your Google Drive (with your labeled data, or the demo data):

### First, place your porject folder into you google drive! "i.e. move the folder named "Project-YourName-TheDate" into google drive.

In [2]:
#Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:
#(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!

Typically, this will be: /content/drive/My Drive/yourProjectFolderName


In [3]:
import os

In [4]:
#Setup your project variables:
# PLEASE EDIT THESE:
  
ProjectFolderName = 'OpenField-Allen-2021-12-01'
VideoType = 'mp4' 

#don't edit these:
videofile_path = ['/content/drive/My Drive/models/dataNeuroMaster/deeplabcut/'+ProjectFolderName+'/videos/'] #Enter the list of videos or folder to analyze.
videofile_path

print("Does this path exist?:",os.path.exists(videofile_path[0]))

Does this path exist?: True


In [5]:
#GUIs don't work on the cloud, so label your data locally on your computer! This will suppress the GUI support
import os
os.environ["DLClight"]="True"

In [6]:
import deeplabcut

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [7]:
deeplabcut.__version__

'2.2.0.3'

In [12]:
#This creates a path variable that links to your google drive copy
#No need to edit this, as you set it up before: 
path_config_file = '/content/drive/My Drive/models/dataNeuroMaster/deeplabcut/'+ProjectFolderName+'/config_COLAB.yaml'
print(path_config_file)

print("Does this path exist?:",os.path.exists(path_config_file))

/content/drive/My Drive/models/dataNeuroMaster/deeplabcut/OpenField-Allen-2021-12-01/config_COLAB.yaml
Does this path exist?: True


## Create a training dataset:
### You must do this step inside of Colab:
After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [ ]:
# Note: if you are using the demo data (i.e. examples/Reaching-Mackenzie-2018-08-30/), first delete the folder called dlc-models! 
#Then, run this cell. There are many more functions you can set here, including which netowkr to use!
#check the docstring for full options you can do!
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.95,
  1,
  (array([17, 20, 23, 10, 25, 19, 26,  8,  1,  7, 13, 30,  0, 35, 21, 12, 15,
           3, 27,  9,  5,  6, 36,  2, 33, 29, 24, 14, 31, 16, 18, 11, 28, 22,
          32, 34]), array([37,  4])))]

## Start training:
This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#if that happens, you can reload from a saved point. Typically, you want to train to 200,000 + iterations.
#more info and there are more things you can set: https://github.com/AlexEMG/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

deeplabcut.train_network(path_config_file, shuffle=1, displayiters=10,saveiters=500)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['nose', 'earL', 'earR', 'tail'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'batch_size': 1,
 'clahe': True,
 'claheratio': 0.1,
 'crop_pad': 0,
 'crop_sampling': 'hybrid',
 'crop_size': [400, 400],
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_OpenFieldDez1/OpenField_Allen95shuffle1.mat',
 'dataset_type': 'imgaug',
 'decay_steps': 30000,
 'deterministic': False,
 'display_iters': 1000,
 'edge': False,
 'emboss': {'alpha': [0.0, 1.0], 'embossratio': 0.1, 'strength': [0.5, 1.5]},
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'histeq': True,
 'histeqratio': 0.1,
 'init_weights': '/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log

Selecting single-animal trainer
Batch Size is 1
Loading ImageNet-pretrained resnet_50
Display_iters overwritten as 10
Save_iters overwritten as 500
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/content/drive/My Drive/models/dataNeuroMaster/deeplabcut/OpenField-Allen-2021-12-01/dlc-models/iteration-0/OpenFieldDez1-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'imgaug', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predic

Streaming output truncated to the last 5000 lines.
iteration: 25260 loss: 0.0037 lr: 0.02
iteration: 25270 loss: 0.0031 lr: 0.02
iteration: 25280 loss: 0.0031 lr: 0.02
iteration: 25290 loss: 0.0035 lr: 0.02
iteration: 25300 loss: 0.0034 lr: 0.02
iteration: 25310 loss: 0.0037 lr: 0.02
iteration: 25320 loss: 0.0030 lr: 0.02
iteration: 25330 loss: 0.0033 lr: 0.02
iteration: 25340 loss: 0.0037 lr: 0.02
iteration: 25350 loss: 0.0033 lr: 0.02
iteration: 25360 loss: 0.0027 lr: 0.02
iteration: 25370 loss: 0.0032 lr: 0.02
iteration: 25380 loss: 0.0036 lr: 0.02
iteration: 25390 loss: 0.0038 lr: 0.02
iteration: 25400 loss: 0.0039 lr: 0.02
iteration: 25410 loss: 0.0037 lr: 0.02
iteration: 25420 loss: 0.0035 lr: 0.02
iteration: 25430 loss: 0.0039 lr: 0.02
iteration: 25440 loss: 0.0033 lr: 0.02
iteration: 25450 loss: 0.0028 lr: 0.02
iteration: 25460 loss: 0.0027 lr: 0.02
iteration: 25470 loss: 0.0033 lr: 0.02
iteration: 25480 loss: 0.0042 lr: 0.02
iteration: 25490 loss: 0.0043 lr: 0.02
iteration: 25

**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating:
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [13]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file,plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, 
#so be sure your labels are good! (And you have trained enough ;)

Running  DLC_resnet50_OpenFieldDez1shuffle1_75000  with # of training iterations: 75000
This net has already been evaluated!
Plotting...(attention scale might be inconsistent in comparison to when data was analyzed; i.e. if you used rescale)


<IPython.core.display.Javascript object>

100%|██████████| 38/38 [00:06<00:00,  5.80it/s]


## There is an optional refinement step you can do outside of Colab:
- if your pixel errors are not low enough, please check out the protocol guide on how to refine your network!
- You will need to adjust the labels **outside of Colab!** We recommend coming back to train and analyze videos... 
- Please see the repo and protocol instructions on how to refine your data!

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
deeplabcut.analyze_videos?

In [14]:
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype="avi")

Using snapshot-75000 for model /content/drive/My Drive/models/dataNeuroMaster/deeplabcut/OpenField-Allen-2021-12-01/dlc-models/iteration-0/OpenFieldDez1-trainset95shuffle1
Analyzing all the videos in the directory...
Starting to analyze %  /content/drive/My Drive/models/dataNeuroMaster/deeplabcut/OpenField-Allen-2021-12-01/videos/mn4673-01102019-1951_trial_57.avi
Starting to analyze %  /content/drive/My Drive/models/dataNeuroMaster/deeplabcut/OpenField-Allen-2021-12-01/videos/mn4673-02102019-1807_trial_65.avi
Starting to analyze %  /content/drive/My Drive/models/dataNeuroMaster/deeplabcut/OpenField-Allen-2021-12-01/videos/mn4673-02102019-1807_trial_64.avi
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_resnet50_OpenFieldDez1shuffle1_75000'

In [19]:
# convert to csv files
deeplabcut.analyze_videos_converth5_to_csv(videofile_path[0],videotype="avi")

['/content/drive/My Drive/models/dataNeuroMaster/deeplabcut/OpenField-Allen-2021-12-01/videos/']
Found output file for scorer: DLC_resnet50_OpenFieldDez1shuffle1_75000
Converting /content/drive/My Drive/models/dataNeuroMaster/deeplabcut/OpenField-Allen-2021-12-01/videos/mn4673-02102019-1807_trial_64DLC_resnet50_OpenFieldDez1shuffle1_75000.h5...
Found output file for scorer: DLC_resnet50_OpenFieldDez1shuffle1_75000
Converting /content/drive/My Drive/models/dataNeuroMaster/deeplabcut/OpenField-Allen-2021-12-01/videos/mn4673-01102019-1951_trial_57DLC_resnet50_OpenFieldDez1shuffle1_75000.h5...
Found output file for scorer: DLC_resnet50_OpenFieldDez1shuffle1_75000
Converting /content/drive/My Drive/models/dataNeuroMaster/deeplabcut/OpenField-Allen-2021-12-01/videos/mn4673-02102019-1807_trial_65DLC_resnet50_OpenFieldDez1shuffle1_75000.h5...
All pose files were converted.


## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
deeplabcut.plot_trajectories(path_config_file,videofile_path, videotype="avi")

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4. 

## Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video?


In [ ]:
print(path_config_file)

/content/drive/My Drive/models/dataNeuroMaster/deeplabcut/OpenField-Allen-2021-12-01/config_COLAB.yaml


In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path, videotype="avi")

Analyzing all the videos in the directory...
Starting to process video: /content/drive/My Drive/models/dataNeuroMaster/deeplabcut/OpenField-Allen-2021-12-01/videos/mn4673-01102019-1951_trial_57.avi
Starting to process video: /content/drive/My Drive/models/dataNeuroMaster/deeplabcut/OpenField-Allen-2021-12-01/videos/mn4673-02102019-1807_trial_64.avi
Loading /content/drive/My Drive/models/dataNeuroMaster/deeplabcut/OpenField-Allen-2021-12-01/videos/mn4673-02102019-1807_trial_64.avi and data.
Loading /content/drive/My Drive/models/dataNeuroMaster/deeplabcut/OpenField-Allen-2021-12-01/videos/mn4673-01102019-1951_trial_57.avi and data.
Duration of video [s]: 13.5, recorded with 30.0 fps!
Overall # of frames: 405 with cropped frame dimensions: 480 480
Generating frames and creating video.


  0%|          | 0/405 [00:00<?, ?it/s]

Duration of video [s]: 12.5, recorded with 30.0 fps!
Overall # of frames: 375 with cropped frame dimensions: 480 480
Generating frames and creating video.


100%|██████████| 375/375 [00:02<00:00, 186.12it/s]


Starting to process video: /content/drive/My Drive/models/dataNeuroMaster/deeplabcut/OpenField-Allen-2021-12-01/videos/mn4673-02102019-1807_trial_65.avi
Loading /content/drive/My Drive/models/dataNeuroMaster/deeplabcut/OpenField-Allen-2021-12-01/videos/mn4673-02102019-1807_trial_65.avi and data.


100%|██████████| 405/405 [00:02<00:00, 193.22it/s]


Duration of video [s]: 13.67, recorded with 30.0 fps!
Overall # of frames: 410 with cropped frame dimensions: 480 480
Generating frames and creating video.


100%|██████████| 410/410 [00:01<00:00, 302.15it/s]
